Folder Path and libraries

In [ ]:
import pandas as pd
import re
import os
import pandas as pd
import datetime

#CONST

folder_path = r"C:\Users\david\My Drive\data\analysis_git_data\cgiar"

1. Adaptation Fund

Add the project number to the original CSV file downloaded from the 'adaptation_fund_path' website. Also, identify projects listed on the website that have no corresponding documents in 'AF_projects_not_in_documents'. Date files indicate when the data collection. 

//TODO : Check with Cesare if AF_projects_not_in_documents needed for the analysis

In [21]:
import pandas as pd
import datetime

# Paths to your CSV files

adaptation_fund_path = os.path.join(folder_path, "scraping_processing_stage", "AF_20240403.csv")  #the file downloaded as csv from ADF
project_documents_path = os.path.join(folder_path, "scraping_processing_stage", "AF_project_documents_links_with_titles_unique.csv")



# Read the CSV files
df_adaptation_fund = pd.read_csv(adaptation_fund_path, on_bad_lines='skip')
df_project_documents = pd.read_csv(project_documents_path, on_bad_lines='skip')

# Ensure 'project_link' is in df_project_documents to proceed with the merge
df_joined = pd.merge(df_adaptation_fund, df_project_documents[['project_link', 'project_title', 'document_link', 'project_no']], left_on='projecturl', right_on='project_link', how='left')

    # Remove the 'project_link' column
df_joined.drop('project_link', axis=1, inplace=True)


current_time = datetime.datetime.now().strftime("%Y%m%d")
#find if projects no documents
projects_not_in_documents = df_adaptation_fund[~df_adaptation_fund['projecturl'].isin(df_project_documents['project_link'])]
filename = f"AF_projects_not_in_documents_{current_time}.csv"
projects_not_in_documents.to_csv(rf"C:\Users\david\My Drive\data\analysis_git_data\cgiar\{filename}", index=False)

# drop if no project number

df_joined_notna = df_joined[df_joined['project_no'].notnull()]
#add org column for master file
df_joined_notna['org'] = 'AF'
df_joined_notna.rename(columns={'projecturl': 'url', 'grantamount':'grant_amount', 'projecttitle': 'title', 'ie':'entity'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  

current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"AF_joined_projects_{current_time}.csv"

full_path = os.path.join(folder_path, filename)

df_joined_notna.to_csv(full_path, index=False)





C:\Users\david\AppData\Local\Temp\ipykernel_20324\4135158987.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_notna['org'] = 'AF'
C:\Users\david\AppData\Local\Temp\ipykernel_20324\4135158987.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_notna.rename(columns={'projecturl': 'url', 'grantamount':'grant_amount', 'projecttitle': 'title', 'ie':'entity'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.


2. GCF 

In [ ]:


gcf_projects = pd.read_csv(os.path.join(folder_path, "scraping_processing_stage", "gcf_projects_20240403.csv")) #the file downloaded as csv from GCF
gcf_projects['org'] = 'GCF'
gcf_projects.rename(columns={'Ref #': 'project_no', 'Countries': 'country', 'Project Name': 'title', 'Sector': 'sector', 'Entity': 'entity', 'BM': 'status', 'FA Financing': 'grant_amount'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  


current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GCF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)

gcf_projects.to_csv(file_path, index=False)


3. GEF

In [ ]:
gef_df = pd.read_csv(os.path.join(folder_path, "gef_projects.csv")) #the file downloaded as csv from GCF
gef_df['org'] = 'GEF'
gef_df.rename(columns={'ID': 'project_no', 'Countries': 'country', 'Title': 'title', 'Focal Areas': 'sector', 'Agencies': 'entity', 'GEF Grant': 'grant_amount', 'Status':'status'}, inplace=True) #rename for consistency with master

current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GEF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)
gef_df.to_csv(file_path, index=False)


Creation of the Master file

In [16]:

af_df = pd.read_csv(os.path.join(folder_path, "AF_joined_projects_20240403.csv"))
gcf_df = pd.read_csv(os.path.join(folder_path, "GCF_joined_projects_20240403.csv"))
gef_df = pd.read_csv(os.path.join(folder_path, "GEF_joined_projects_20240403.csv"))

af_df['project_no'] = af_df['project_no'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)
gcf_df['project_no'] = gcf_df['project_no'].astype(str)
gef_df['project_no'] = gef_df['project_no'].apply(lambda x: str(int(x)) if pd.notnull(x) else None)

all_projects_df = pd.concat([af_df, gcf_df, gef_df], ignore_index=True)

#order df
desired_order = ['project_no', 'org', 'country', 'title', 'sector', 'entity', 'status', 'grant_amount']
additional_columns = [col for col in all_projects_df.columns if col not in desired_order]
final_column_order = desired_order + additional_columns

#if you want all column add final_column_order instead of desired_order

all_projects_df = all_projects_df[desired_order]

current_time = datetime.datetime.now().strftime("%Y%m%d")

filename = f"master_file_af_gcf_gef_{current_time}.csv"

#save as csv
all_projects_df.to_csv(os.path.join(folder_path, filename), index=False)



#CHECK final projects with website CSV content
print(f"Total projects from the Adaptation Fund are {len(af_df)}. On the website, there are 149. Missing or not in dataset: {149 - len(af_df)}.")
print(f"Total projects from the GCF are {len(gcf_df)}. On the website, there are 253. Missing or not in dataset: {253 - len(gcf_df)}.")
print(f"Total projects from the GEF are {len(gef_df)}. On the website, there are 6079. Missing or not in dataset: {6079 - len(gef_df)}.")


Total projects from the Adaptation Fund are 147. On the website, there are 149. Missing or not in dataset: 2.
Total projects from the GCF are 253. On the website, there are 253. Missing or not in dataset: 0.
Total projects from the GEF are 6079. On the website, there are 6079. Missing or not in dataset: 0.
